In [1]:
# requires vdf>=1.6
import vdf

class DuplicateOrderedDict(dict):
    def __init__(self, *args, **kwds):
        if len(args) > 1:
            raise TypeError('expected at most 1 arguments, got %d' % len(args))
        self.__omap = []
        if args:
            if not isinstance(args[0], list):
                raise TypeError("Expected a list of (key, value) tuples, got %s" % type(args[0]))
                
            for pair in args[0]:
                if not isinstance(pair, tuple) and len(pair) != 2:
                    raise TypeError("Expected a list of (key, value) tuples, got %s" % type(args[0]))
                self.__setitem__(*pair)
                
    def __repr__(self):
        out = "%s(" % self.__class__.__name__
        out += "%s)" % repr(self.items())
        return out
    
    def __setitem__(self, key, value):
        if not isinstance(key, tuple):
            idx = 0
            while True:
                if (idx, key) not in self:
                    self.__omap.append((idx, key))
                    break
                else:
                    idx += 1
            key = (idx, key)
        else:
            if key not in self:
                raise KeyError("%s doesn't exist" % repr(key))
        
        super(DuplicateOrderedDict, self).__setitem__(key, value)
        
    def __getitem__(self, key):
        if not isinstance(key, tuple):
            key = (0, key)
        
        return super(DuplicateOrderedDict, self).__getitem__(key)
    
    def __iter__(self):
        return iter(map(lambda x: x[1], self.__omap))
    
    def items(self):
        return [(key[1], self[key]) for key in self.__omap]

In [2]:
data = vdf.load(open('npc_heroes.txt'), mapper=DuplicateOrderedDict)
dups = data['DOTAHeroes']['npc_dota_hero_zuus']['Bot']['Loadout']

In [3]:
dups

DuplicateOrderedDict([('item_tango', 'ITEM_CONSUMABLE | ITEM_SELLABLE'), ('item_clarity', 'ITEM_CONSUMABLE | ITEM_SELLABLE'), ('item_clarity', 'ITEM_CONSUMABLE | ITEM_SELLABLE'), ('item_mantle', 'ITEM_CORE'), ('item_circlet', 'ITEM_CORE'), ('item_recipe_null_talisman', 'ITEM_CORE'), ('item_null_talisman', 'ITEM_DERIVED | ITEM_SELLABLE'), ('item_boots', 'ITEM_CORE'), ('item_magic_stick', 'ITEM_CORE | ITEM_SELLABLE'), ('item_point_booster', 'ITEM_CORE'), ('item_staff_of_wizardry', 'ITEM_CORE'), ('item_ogre_axe', 'ITEM_CORE'), ('item_blade_of_alacrity', 'ITEM_CORE'), ('item_ultimate_scepter', 'ITEM_DERIVED'), ('item_blades_of_attack', 'ITEM_CORE'), ('item_blades_of_attack', 'ITEM_CORE'), ('item_phase_boots', 'ITEM_DERIVED'), ('item_void_stone', 'ITEM_EXTENSION'), ('item_ring_of_health', 'ITEM_EXTENSION'), ('item_pers', 'ITEM_DERIVED'), ('item_sobi_mask', 'ITEM_EXTENSION'), ('item_robe', 'ITEM_EXTENSION'), ('item_quarterstaff', 'ITEM_EXTENSION'), ('item_oblivion_staff', 'ITEM_DERIVED'), ('

In [4]:
dups.items()

[('item_tango', 'ITEM_CONSUMABLE | ITEM_SELLABLE'),
 ('item_clarity', 'ITEM_CONSUMABLE | ITEM_SELLABLE'),
 ('item_clarity', 'ITEM_CONSUMABLE | ITEM_SELLABLE'),
 ('item_mantle', 'ITEM_CORE'),
 ('item_circlet', 'ITEM_CORE'),
 ('item_recipe_null_talisman', 'ITEM_CORE'),
 ('item_null_talisman', 'ITEM_DERIVED | ITEM_SELLABLE'),
 ('item_boots', 'ITEM_CORE'),
 ('item_magic_stick', 'ITEM_CORE | ITEM_SELLABLE'),
 ('item_point_booster', 'ITEM_CORE'),
 ('item_staff_of_wizardry', 'ITEM_CORE'),
 ('item_ogre_axe', 'ITEM_CORE'),
 ('item_blade_of_alacrity', 'ITEM_CORE'),
 ('item_ultimate_scepter', 'ITEM_DERIVED'),
 ('item_blades_of_attack', 'ITEM_CORE'),
 ('item_blades_of_attack', 'ITEM_CORE'),
 ('item_phase_boots', 'ITEM_DERIVED'),
 ('item_void_stone', 'ITEM_EXTENSION'),
 ('item_ring_of_health', 'ITEM_EXTENSION'),
 ('item_pers', 'ITEM_DERIVED'),
 ('item_sobi_mask', 'ITEM_EXTENSION'),
 ('item_robe', 'ITEM_EXTENSION'),
 ('item_quarterstaff', 'ITEM_EXTENSION'),
 ('item_oblivion_staff', 'ITEM_DERIVED'),


In [5]:
# lets start from scratch
dups = DuplicateOrderedDict([
        ("item_tango", "ASD"),
        ("item_tango", "ZXC"),
        ("item_clarity", "CONSUMABLE"),
        ("item_tango", "123"),
        ("item_tango", "456"),
])
dups.items()

[('item_tango', 'ASD'),
 ('item_tango', 'ZXC'),
 ('item_clarity', 'CONSUMABLE'),
 ('item_tango', '123'),
 ('item_tango', '456')]

In [6]:
DuplicateOrderedDict(dups.items()).items()

[('item_tango', 'ASD'),
 ('item_tango', 'ZXC'),
 ('item_clarity', 'CONSUMABLE'),
 ('item_tango', '123'),
 ('item_tango', '456')]

In [7]:
# it's just a dict with tuple for keys
dict(dups)

{(0, 'item_clarity'): 'CONSUMABLE',
 (0, 'item_tango'): 'ASD',
 (1, 'item_tango'): 'ZXC',
 (2, 'item_tango'): '123',
 (3, 'item_tango'): '456'}

In [8]:
# reverse order, or alternatively implement DuplicateOrderedDict.reverse()
DuplicateOrderedDict(list(reversed(dups.items()))).items()

[('item_tango', '456'),
 ('item_tango', '123'),
 ('item_clarity', 'CONSUMABLE'),
 ('item_tango', 'ZXC'),
 ('item_tango', 'ASD')]

In [9]:
print dups['item_tango']
print dups[(0, 'item_tango')] # same as above
print dups[(1, 'item_tango')] # first duplicate
print dups[(10, 'item_tango')] # doesnt exist :(

ASD
ASD
ZXC


KeyError: (10, 'item_tango')

In [10]:
# new duplicate keys are appended at the end
dups['item_clarity'] = "NEW DUPLICATE"
# to reassign a value use tuple key
dups[(0, 'item_tango')] = "NEW VALUE"
dups.items()

[('item_tango', 'NEW VALUE'),
 ('item_tango', 'ZXC'),
 ('item_clarity', 'CONSUMABLE'),
 ('item_tango', '123'),
 ('item_tango', '456'),
 ('item_clarity', 'NEW DUPLICATE')]

In [11]:
data['DOTAHeroes']['npc_dota_hero_zuus']['Bot'][(0,'Loadout')] = dups
print vdf.dumps(data['DOTAHeroes']['npc_dota_hero_zuus']['Bot'], pretty=True)

"Loadout"
{
	"item_tango" "NEW VALUE"
	"item_tango" "ZXC"
	"item_clarity" "CONSUMABLE"
	"item_tango" "123"
	"item_tango" "456"
	"item_clarity" "NEW DUPLICATE"
}
"Build"
{
	"1" "zuus_lightning_bolt"
	"2" "zuus_static_field"
	"3" "zuus_lightning_bolt"
	"4" "zuus_arc_lightning"
	"5" "zuus_lightning_bolt"
	"6" "zuus_thundergods_wrath"
	"7" "zuus_lightning_bolt"
	"8" "zuus_static_field"
	"9" "zuus_static_field"
	"10" "zuus_static_field"
	"11" "zuus_thundergods_wrath"
	"12" "zuus_arc_lightning"
	"13" "zuus_arc_lightning"
	"14" "zuus_arc_lightning"
	"15" "attribute_bonus"
	"16" "zuus_thundergods_wrath"
	"17" "attribute_bonus"
	"18" "attribute_bonus"
	"19" "attribute_bonus"
	"20" "attribute_bonus"
	"21" "attribute_bonus"
	"22" "attribute_bonus"
	"23" "attribute_bonus"
	"24" "attribute_bonus"
	"25" "attribute_bonus"
}
"LaningInfo"
{
	"SoloDesire" "1"
	"RequiresBabysit" "0"
	"ProvidesBabysit" "1"
	"SurvivalRating" "0"
	"RequiresFarm" "1"
	"ProvidesSetup" "0"
	"RequiresSetup" "1"
}
"HeroType" "DO